## Ddip User Guide 
### `%dipush` and `%dipull` --- Pass objects between local notebook and DDP processes

In [1]:
%load_ext Ddip
%makedip -r -g all --verbose False

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-ippdpp_c-client.json


In [2]:
%%dip everywhere
import os # For convenience


After creating a 'remote' pool of process for DDP, we are still in local notebook namespace.
Two ways to get variables over to from local to remote:

1. Create locally, then %dipush.

2. Create simultaneously with %%dip everywhere

**The first way first:**


In [3]:
foobar = { 1 : "foo", 2 : "bar" }
%dipush foobar

In [4]:
%%dip everywhere --see all
print(f"Process [{os.getpid()}] foobar is :{foobar}")

Process [13980] foobar is :{1: 'foo', 2: 'bar'}
Process [14034] foobar is :{1: 'foo', 2: 'bar'}
Process [14038] foobar is :{1: 'foo', 2: 'bar'}
Process [14041] foobar is :{1: 'foo', 2: 'bar'}


**The second way:**

In [5]:
%%dip everywhere
x = [ "Different things", 2, ["a", "list"], {"and": "a dict"}]

In [6]:
# Back in local notebook namespace:
x

['Different things', 2, ['a', 'list'], {'and': 'a dict'}]

In [7]:
%%dip --see all

# and in remote DDP namespace:
x

Out[0:4]: ['Different things', 2, ['a', 'list'], {'and': 'a dict'}]

Out[1:4]: ['Different things', 2, ['a', 'list'], {'and': 'a dict'}]

Out[2:4]: ['Different things', 2, ['a', 'list'], {'and': 'a dict'}]


Create something remotely, then pull back from rank-0 process with `%dipull`

In [8]:
%%dip
remote_x = "I am created remotely"

In [9]:
# Won't see it yet:
remote_x

NameError: name 'remote_x' is not defined

Notice the process ID below verifies it is the local process,

In [10]:
%dipull remote_x
print(f"Process [{os.getpid()}] remote_x is :{remote_x}")

Process [13980] remote_x is :I am created remotely
